# Task 1.4: Compatibility Matrix

**Module:** 1 - DGX Spark Platform Mastery  
**Time:** 2 hours  
**Difficulty:** ⭐⭐ (Intermediate)

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- [ ] Understand why some tools don't work on DGX Spark
- [ ] Research and document 20+ AI/ML tool compatibility
- [ ] Learn workarounds for partially supported tools
- [ ] Create a comprehensive reference matrix

---

## 📚 Prerequisites

- Completed: Task 1.1-1.3
- Basic understanding of ARM64 architecture
- Familiarity with common AI/ML tools

---

## 🌍 Real-World Context

You've just received your DGX Spark. Excitedly, you try to install your favorite tools:

```bash
pip install torch  # ❌ Fails - wrong architecture
pip install vllm   # ❌ Fails - x86_64 only wheels
```

This frustration is common! The DGX Spark uses ARM64 (aarch64) architecture with a cutting-edge Blackwell GPU. Many tools simply haven't caught up yet.

This notebook helps you navigate the ecosystem and find what works.

---

## 🧒 ELI5: Why Don't Some Tools Work?

> **Imagine you have a special game console that plays a new type of game disc...**
>
> Most games are made for the old disc format (x86). Your new console (DGX Spark with ARM64) needs games made specifically for it.
>
> Some game makers have already made new versions (NVIDIA with NGC containers). Others are still working on it. And some old games might never get ported.
>
> **In AI terms:** Software must be compiled for ARM64 + CUDA 13 + Blackwell GPU. This is a rare combination that many projects don't support yet.

---

## Part 1: Understanding Compatibility Challenges

### Why Things Break

Three main reasons why tools may not work on DGX Spark:

1. **Architecture (ARM64 vs x86_64)**
   - Most Python wheels are pre-compiled for x86_64
   - ARM64 requires separate compilation
   - Many projects don't publish ARM64 wheels

2. **CUDA Version (13.x)**
   - DGX Spark uses CUDA 13.0+
   - Many tools only support CUDA 11.x or 12.x
   - CUDA versions are NOT backward compatible

3. **GPU Architecture (Blackwell)**
   - Blackwell (SM 100) is brand new
   - Older CUDA code may not include Blackwell support
   - Some optimizations require architecture-specific code

In [ ]:
# Let's verify our platform details
import platform
import subprocess

print("DGX Spark Platform Details")
print("=" * 50)
print(f"Architecture: {platform.machine()}")
print(f"OS: {platform.system()} {platform.release()}")
print(f"Python: {platform.python_version()}")

# Get CUDA version
try:
    result = subprocess.run(["nvidia-smi"], capture_output=True, text=True)
    for line in result.stdout.split('\n'):
        if 'CUDA Version' in line:
            cuda_ver = line.split('CUDA Version:')[1].split()[0]
            print(f"CUDA: {cuda_ver}")
            break
except Exception:
    print("CUDA: Unable to determine")

---

## Part 2: Compatibility Categories

We'll categorize tools into these support levels:

| Level | Icon | Meaning |
|-------|------|----------|
| Full Support | ✅ | Works out of the box |
| NGC Required | 🐳 | Works via NGC containers |
| Partial | ⚠️ | Works with limitations/workarounds |
| Not Compatible | ❌ | Does not work currently |
| Untested | ❓ | Needs testing |

In [ ]:
# Define compatibility data structure
from dataclasses import dataclass
from typing import Optional, List
from enum import Enum

class CompatLevel(Enum):
    FULL = "✅ Full Support"
    NGC = "🐳 NGC Required"
    PARTIAL = "⚠️ Partial"
    NO = "❌ Not Compatible"
    UNTESTED = "❓ Untested"

@dataclass
class ToolCompatibility:
    """Compatibility information for an AI/ML tool."""
    name: str
    category: str
    level: CompatLevel
    notes: str
    workaround: Optional[str] = None
    tested_version: Optional[str] = None
    url: Optional[str] = None

print("Compatibility data structure defined!")

---

## Part 3: The Compatibility Matrix

Here's the comprehensive compatibility matrix for DGX Spark. This is based on testing and community reports as of 2025.

In [ ]:
# Comprehensive compatibility database

compatibility_matrix = [
    # ========== Deep Learning Frameworks ==========
    ToolCompatibility(
        name="PyTorch",
        category="Deep Learning Framework",
        level=CompatLevel.NGC,
        notes="Must use NGC container. pip install does NOT work on ARM64+CUDA.",
        workaround="docker pull nvcr.io/nvidia/pytorch:25.11-py3",
        tested_version="2.5+",
        url="https://catalog.ngc.nvidia.com/orgs/nvidia/containers/pytorch"
    ),
    ToolCompatibility(
        name="TensorFlow",
        category="Deep Learning Framework",
        level=CompatLevel.NGC,
        notes="Use NGC container for GPU support.",
        workaround="docker pull nvcr.io/nvidia/tensorflow:25.11-tf2-py3",
        tested_version="2.16+"
    ),
    ToolCompatibility(
        name="JAX",
        category="Deep Learning Framework",
        level=CompatLevel.NGC,
        notes="NGC container available with CUDA support.",
        workaround="docker pull nvcr.io/nvidia/jax:25.11-py3"
    ),
    
    # ========== LLM Inference ==========
    ToolCompatibility(
        name="Ollama",
        category="LLM Inference",
        level=CompatLevel.FULL,
        notes="Native ARM64 support. Pre-installed on DGX OS. Excellent performance.",
        tested_version="0.3+",
        url="https://ollama.ai"
    ),
    ToolCompatibility(
        name="llama.cpp",
        category="LLM Inference",
        level=CompatLevel.FULL,
        notes="Native ARM64+CUDA support. Compile with CUDA flags.",
        workaround="cmake -B build -DGGML_CUDA=ON && cmake --build build",
        url="https://github.com/ggerganov/llama.cpp"
    ),
    ToolCompatibility(
        name="vLLM",
        category="LLM Inference",
        level=CompatLevel.PARTIAL,
        notes="Experimental ARM64 support. May require building from source.",
        workaround="Build from source with ARM64+CUDA flags",
        url="https://github.com/vllm-project/vllm"
    ),
    ToolCompatibility(
        name="TensorRT-LLM",
        category="LLM Inference",
        level=CompatLevel.PARTIAL,
        notes="NVIDIA product - Blackwell support in development.",
        workaround="Use NGC container with latest TRT-LLM"
    ),
    ToolCompatibility(
        name="Text Generation Inference (TGI)",
        category="LLM Inference",
        level=CompatLevel.PARTIAL,
        notes="HuggingFace server. ARM64 Docker image available.",
        workaround="Use ARM64 Docker image"
    ),
    
    # ========== Model Libraries ==========
    ToolCompatibility(
        name="Hugging Face Transformers",
        category="Model Library",
        level=CompatLevel.NGC,
        notes="Works inside NGC PyTorch container.",
        workaround="pip install transformers inside NGC container",
        tested_version="4.40+"
    ),
    ToolCompatibility(
        name="Hugging Face Diffusers",
        category="Model Library",
        level=CompatLevel.NGC,
        notes="Works inside NGC PyTorch container.",
        workaround="pip install diffusers inside NGC container"
    ),
    ToolCompatibility(
        name="NVIDIA NeMo",
        category="Model Library",
        level=CompatLevel.NGC,
        notes="Full support via NGC container.",
        workaround="docker pull nvcr.io/nvidia/nemo:25.11",
        url="https://catalog.ngc.nvidia.com/orgs/nvidia/containers/nemo"
    ),
    ToolCompatibility(
        name="OpenAI API (client)",
        category="Model Library",
        level=CompatLevel.FULL,
        notes="Pure Python - works everywhere.",
        tested_version="1.0+"
    ),
    ToolCompatibility(
        name="LangChain",
        category="Model Library",
        level=CompatLevel.FULL,
        notes="Pure Python - works everywhere. Use with Ollama.",
        tested_version="0.2+"
    ),
    ToolCompatibility(
        name="LlamaIndex",
        category="Model Library",
        level=CompatLevel.FULL,
        notes="Pure Python - works everywhere.",
        tested_version="0.10+"
    ),
    
    # ========== Training & Fine-tuning ==========
    ToolCompatibility(
        name="DeepSpeed",
        category="Training",
        level=CompatLevel.PARTIAL,
        notes="Some features may not work. Use NGC container.",
        workaround="Use within NGC PyTorch container"
    ),
    ToolCompatibility(
        name="PEFT (LoRA)",
        category="Training",
        level=CompatLevel.NGC,
        notes="Works inside NGC container.",
        workaround="pip install peft inside NGC container"
    ),
    ToolCompatibility(
        name="bitsandbytes",
        category="Training",
        level=CompatLevel.PARTIAL,
        notes="4-bit/8-bit quantization. ARM64 support improving.",
        workaround="May need to build from source"
    ),
    ToolCompatibility(
        name="Unsloth",
        category="Training",
        level=CompatLevel.UNTESTED,
        notes="Fast fine-tuning. Needs testing on ARM64."
    ),
    ToolCompatibility(
        name="Axolotl",
        category="Training",
        level=CompatLevel.NGC,
        notes="Fine-tuning framework. Use with NGC container."
    ),
    
    # ========== Data Science ==========
    ToolCompatibility(
        name="RAPIDS (cuDF, cuML)",
        category="Data Science",
        level=CompatLevel.NGC,
        notes="GPU-accelerated data science. Full NGC support.",
        workaround="docker pull nvcr.io/nvidia/rapidsai/base:25.11-cuda13.0-py3.11"
    ),
    ToolCompatibility(
        name="Pandas",
        category="Data Science",
        level=CompatLevel.FULL,
        notes="Pure Python - works everywhere."
    ),
    ToolCompatibility(
        name="NumPy",
        category="Data Science",
        level=CompatLevel.FULL,
        notes="ARM64 wheels available."
    ),
    ToolCompatibility(
        name="Scikit-learn",
        category="Data Science",
        level=CompatLevel.FULL,
        notes="ARM64 wheels available."
    ),
    
    # ========== Vector Databases ==========
    ToolCompatibility(
        name="ChromaDB",
        category="Vector Database",
        level=CompatLevel.FULL,
        notes="Pure Python with SQLite. Works everywhere."
    ),
    ToolCompatibility(
        name="FAISS",
        category="Vector Database",
        level=CompatLevel.NGC,
        notes="GPU version needs NGC container.",
        workaround="Use faiss-gpu inside NGC container"
    ),
    ToolCompatibility(
        name="Milvus",
        category="Vector Database",
        level=CompatLevel.PARTIAL,
        notes="ARM64 Docker images available."
    ),
    ToolCompatibility(
        name="Qdrant",
        category="Vector Database",
        level=CompatLevel.FULL,
        notes="ARM64 Docker images available."
    ),
    
    # ========== Development Tools ==========
    ToolCompatibility(
        name="JupyterLab",
        category="Development",
        level=CompatLevel.FULL,
        notes="Pre-installed on DGX OS."
    ),
    ToolCompatibility(
        name="VS Code",
        category="Development",
        level=CompatLevel.FULL,
        notes="ARM64 version available."
    ),
    ToolCompatibility(
        name="Docker",
        category="Development",
        level=CompatLevel.FULL,
        notes="Pre-installed with NVIDIA runtime."
    ),
    ToolCompatibility(
        name="Git",
        category="Development",
        level=CompatLevel.FULL,
        notes="Pre-installed."
    ),
    
    # ========== MLOps ==========
    ToolCompatibility(
        name="MLflow",
        category="MLOps",
        level=CompatLevel.FULL,
        notes="Pure Python - works everywhere."
    ),
    ToolCompatibility(
        name="Weights & Biases",
        category="MLOps",
        level=CompatLevel.FULL,
        notes="Pure Python - works everywhere."
    ),
    ToolCompatibility(
        name="Triton Inference Server",
        category="MLOps",
        level=CompatLevel.NGC,
        notes="Full NGC support.",
        workaround="docker pull nvcr.io/nvidia/tritonserver:25.11-py3"
    ),
]

print(f"Loaded {len(compatibility_matrix)} tools into compatibility matrix.")

---

## Part 4: Viewing the Matrix

In [ ]:
# Display the compatibility matrix

def display_matrix_by_category():
    """Display the compatibility matrix organized by category."""
    categories = {}
    for tool in compatibility_matrix:
        if tool.category not in categories:
            categories[tool.category] = []
        categories[tool.category].append(tool)
    
    for category, tools in categories.items():
        print(f"\n{'=' * 60}")
        print(f"  {category}")
        print(f"{'=' * 60}")
        print(f"{'Tool':<30} {'Status':<20}")
        print("-" * 50)
        for tool in tools:
            print(f"{tool.name:<30} {tool.level.value:<20}")

display_matrix_by_category()

In [ ]:
# Create a summary table
def create_summary():
    """Create a summary of compatibility levels."""
    counts = {level: 0 for level in CompatLevel}
    for tool in compatibility_matrix:
        counts[tool.level] += 1
    
    print("\n" + "=" * 50)
    print("COMPATIBILITY SUMMARY")
    print("=" * 50)
    total = len(compatibility_matrix)
    
    for level, count in counts.items():
        pct = count / total * 100
        bar = "█" * int(pct / 5) + "░" * (20 - int(pct / 5))
        print(f"{level.value:<20} [{bar}] {count:>2} ({pct:.0f}%)")
    
    print(f"\nTotal tools evaluated: {total}")

create_summary()

---

## Part 5: Detailed Tool Lookup

In [ ]:
def lookup_tool(name: str):
    """Look up detailed information for a specific tool."""
    for tool in compatibility_matrix:
        if name.lower() in tool.name.lower():
            print(f"\n{'=' * 60}")
            print(f"  {tool.name}")
            print(f"{'=' * 60}")
            print(f"Category:    {tool.category}")
            print(f"Status:      {tool.level.value}")
            print(f"Notes:       {tool.notes}")
            if tool.workaround:
                print(f"Workaround:  {tool.workaround}")
            if tool.tested_version:
                print(f"Tested Ver:  {tool.tested_version}")
            if tool.url:
                print(f"URL:         {tool.url}")
            return
    print(f"Tool '{name}' not found in matrix.")

# Example lookups
lookup_tool("PyTorch")
lookup_tool("Ollama")
lookup_tool("vLLM")

### ✋ Try It Yourself #1

Look up a tool you use frequently. Is it compatible?

<details>
<summary>💡 Hint</summary>
Try: `lookup_tool("transformers")` or `lookup_tool("langchain")`
</details>

In [ ]:
# YOUR CODE HERE: Look up your favorite tool


---

## Part 6: Export the Matrix

In [ ]:
def export_markdown_table():
    """Export compatibility matrix as markdown table."""
    lines = [
        "# DGX Spark Compatibility Matrix\n",
        "Last updated: " + __import__('datetime').datetime.now().strftime("%Y-%m-%d") + "\n",
        "\n| Tool | Category | Status | Notes |",
        "|------|----------|--------|-------|"
    ]
    
    for tool in sorted(compatibility_matrix, key=lambda x: (x.category, x.name)):
        status = tool.level.value
        notes = tool.notes[:50] + "..." if len(tool.notes) > 50 else tool.notes
        lines.append(f"| {tool.name} | {tool.category} | {status} | {notes} |")
    
    return "\n".join(lines)

# Generate and display
markdown_table = export_markdown_table()
print(markdown_table[:2000])  # First 2000 chars

In [ ]:
# Save to file
with open("compatibility_matrix.md", "w") as f:
    f.write(export_markdown_table())

print("✅ Saved to compatibility_matrix.md")

In [ ]:
# Export as JSON for programmatic use
import json
from datetime import datetime

def export_json():
    """Export compatibility matrix as JSON."""
    data = {
        "generated": datetime.now().isoformat(),
        "platform": "NVIDIA DGX Spark",
        "tools": [
            {
                "name": t.name,
                "category": t.category,
                "status": t.level.name,
                "status_display": t.level.value,
                "notes": t.notes,
                "workaround": t.workaround,
                "tested_version": t.tested_version,
                "url": t.url
            }
            for t in compatibility_matrix
        ]
    }
    return data

# Save JSON
with open("compatibility_matrix.json", "w") as f:
    json.dump(export_json(), f, indent=2)

print("✅ Saved to compatibility_matrix.json")

---

## Part 7: Quick Compatibility Checker

Let's create a utility function to quickly check if a package can be installed.

In [ ]:
def check_pip_compatibility(package_name: str) -> dict:
    """
    Check if a pip package has ARM64 Linux wheels.
    
    Returns:
        dict with compatibility info
    """
    import subprocess
    import re
    
    result = {
        "package": package_name,
        "has_arm64_wheel": False,
        "has_any_wheel": False,
        "is_pure_python": False,
        "recommendation": ""
    }
    
    try:
        # Query PyPI for package info
        import urllib.request
        import json
        
        url = f"https://pypi.org/pypi/{package_name}/json"
        with urllib.request.urlopen(url, timeout=10) as response:
            data = json.loads(response.read())
        
        # Check available wheels
        urls = data.get("urls", [])
        
        for url_info in urls:
            filename = url_info.get("filename", "")
            
            if filename.endswith(".whl"):
                result["has_any_wheel"] = True
                
                # Check for ARM64
                if "aarch64" in filename or "arm64" in filename:
                    result["has_arm64_wheel"] = True
                
                # Check for pure Python (py3-none-any)
                if "py3-none-any" in filename or "py2.py3-none-any" in filename:
                    result["is_pure_python"] = True
        
        # Make recommendation
        if result["is_pure_python"]:
            result["recommendation"] = "✅ Pure Python - should work directly"
        elif result["has_arm64_wheel"]:
            result["recommendation"] = "✅ ARM64 wheel available - pip install should work"
        elif result["has_any_wheel"]:
            result["recommendation"] = "⚠️ No ARM64 wheel - may need NGC container or build from source"
        else:
            result["recommendation"] = "❓ No wheels found - check documentation"
            
    except Exception as e:
        result["recommendation"] = f"❌ Error checking package: {e}"
    
    return result

# Test some packages
test_packages = ["numpy", "pandas", "transformers", "torch", "langchain"]

print("Package Compatibility Check")
print("=" * 60)

for pkg in test_packages:
    result = check_pip_compatibility(pkg)
    print(f"\n{result['package']}:")
    print(f"  {result['recommendation']}")

---

## Part 8: Community Resources

Where to find help with compatibility issues:

| Resource | URL | Purpose |
|----------|-----|----------|
| NVIDIA Forums | forums.developer.nvidia.com | Official support |
| NGC Catalog | catalog.ngc.nvidia.com | Container images |
| DGX Playbooks | build.nvidia.com/spark | Guides and examples |
| GitHub Issues | Search "aarch64" or "ARM64" | Community solutions |

### Key Tips for Finding Compatibility Info

1. **Search GitHub Issues:** Many projects have ARM64 compatibility discussions
2. **Check NGC First:** NVIDIA often provides pre-built containers
3. **Look for Docker Images:** Multi-arch images often support ARM64
4. **Check Recent Commits:** ARM64 support may be in development

---

## ⚠️ Common Mistakes

### Mistake 1: Trying pip install for everything

```bash
# ❌ Wrong - Will fail for many packages
pip install torch transformers accelerate

# ✅ Right - Use NGC container first
docker run --gpus all -it nvcr.io/nvidia/pytorch:25.11-py3 bash
# Then inside container:
pip install transformers accelerate
```

### Mistake 2: Assuming x86 Docker images work

```bash
# ❌ Wrong - x86 image won't run on ARM64
docker pull someproject/tool:latest  # May be x86 only!

# ✅ Right - Check for ARM64/multi-arch support
docker manifest inspect someproject/tool:latest | grep arm64
```

### Mistake 3: Not checking NGC before building from source

```bash
# ❌ Wrong - Waste time building
git clone https://github.com/project/tool
pip install -e .  # Hours of compilation...

# ✅ Right - Check NGC first
# Search: catalog.ngc.nvidia.com
docker pull nvcr.io/nvidia/tool:latest  # Done in minutes!
```

---

## 🎉 Checkpoint

You've learned:
- ✅ Why ARM64 + CUDA creates compatibility challenges
- ✅ How to categorize tool compatibility levels
- ✅ Created a comprehensive 30+ tool compatibility matrix
- ✅ How to check pip package ARM64 support
- ✅ Where to find help and workarounds

---

## 🚀 Challenge (Optional)

Add 5 more tools to the compatibility matrix that you use regularly. Research their ARM64/DGX Spark compatibility.

<details>
<summary>💡 Hint</summary>
Check:
- GitHub releases for ARM64 builds
- Docker Hub for multi-arch images
- PyPI for aarch64 wheels
- NGC catalog for NVIDIA-optimized versions
</details>

In [ ]:
# YOUR CODE HERE: Add more tools to the matrix

# Example:
# my_new_tools = [
#     ToolCompatibility(
#         name="My Tool",
#         category="Category",
#         level=CompatLevel.FULL,
#         notes="Works great!"
#     ),
# ]
# compatibility_matrix.extend(my_new_tools)


---

## 📖 Further Reading

- [NGC Container Catalog](https://catalog.ngc.nvidia.com/)
- [PyPI Classifier for Platform](https://pypi.org/classifiers/)
- [Docker Multi-Platform Images](https://docs.docker.com/build/building/multi-platform/)
- [DGX Spark Playbooks](https://build.nvidia.com/spark)

---

## 🧹 Cleanup

In [ ]:
print("Files created in this notebook:")
print("-" * 40)
print("  - compatibility_matrix.md")
print("  - compatibility_matrix.json")
print("\nThese are reference files - keep them!")

print("\n" + "=" * 60)
print("🎉 Great job completing Task 1.4: Compatibility Matrix!")
print("=" * 60)
print("\nNext up: Task 1.5 - Ollama Benchmarking")
print("You'll benchmark LLM inference performance on your DGX Spark.")